In [1]:
# Théo Ryffel [5:16 PM]
import syft as sy
from syft import optim
import sys
import torch
from torch.nn import Parameter
import torch.nn as nn
hook = sy.TorchHook(torch)

import torch.nn.functional as F

me = hook.local_worker
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

In [2]:

# TEST: simple remote grad calculation

# create a tensor
x = torch.tensor([1, 2, 3, 4.0], requires_grad=True)

# send tensor to bob
x = x.send(bob)

# do some calculatinos
y = (x + x).sum()

# send gradient to backprop to Bob
grad = torch.tensor([1.0]).send(bob)

# backpropagate on remote machine
y.backward(grad)

# check that remote gradient is correct
xgrad = bob._objects[x.id_at_location].grad
xgrad_target = torch.ones(4).float() + 1

assert (xgrad == xgrad_target).all()

In [22]:
# create a tensor
x = torch.tensor([1, 2, 3, 4.0], requires_grad=True)

# send tensor to bob
x = x.send(bob)

# do some calculatinos
y = (x + x).sum()

# send gradient to backprop to Bob
grad = torch.tensor([1.0]).send(bob)

# backpropagate on remote machine
y.backward(grad)

In [23]:

# check that remote gradient is correct
xgrad = bob._objects[x.id_at_location].grad
xgrad_target = torch.ones(4).float() + 1

assert (xgrad == xgrad_target).all()

In [24]:
# create a tensor
x = torch.tensor([1, 2, 3, 4.0], requires_grad=True)

# send tensor to bob
x = x.send(bob)

# do some calculatinos
y = (x + x).sum()

# send gradient to backprop to Bob
grad = torch.tensor([1.0]).send(bob)

# backpropagate on remote machine
y.backward(grad)

# check that remote gradient is correct
xgrad = bob._objects[x.id_at_location].grad
xgrad_target = torch.ones(4).float() + 1

assert (xgrad == xgrad_target).all()

# TEST: Ensure remote grad calculation gets properly serded

# create tensor
x = torch.tensor([1, 2, 3, 4.0], requires_grad=True).send(bob)

# create output gradient
out_grad = torch.tensor([1.0]).send(bob)

# compute function
y = x.sum()

# backpropagate
y.backward(out_grad)

# get the gradient created from backpropagation manually
x_grad = bob._objects[x.id_at_location].grad

# get the entire x tensor (should bring the grad too)
x = x.get()

# make sure that the grads match
assert (x.grad == x_grad).all()